Importing Libraries : 
- **random** library is used to sample random number for the key generation.
- **Crypto.Util** is used to sample a prime number for the OWP.

In [2]:
import random
from Crypto.Util import number


Global variables
- *length* is the length of maximum prime that can be sampled. It can be kept small for the code to run faster. 

In [3]:
p = 0 
length = 8 
Key = None
gp = None

# Functions
- **start_session()** is used to sample the prime number for the OWP.
- **sieve(int)** is used to sieve out the primes from the other numbers.
- **generator()** is used to get the generator for the group used in OWP
- **owp(str)** is used to send the OWP value corresponding to the input which is binary string format
- **innerproduct(str,str)** is used to calculate the value of inner product of the 2 binary strings
- **PRG_1(str)** is a PRG which increases the length of the output by 1 where the key is still a binary string.
- **PRG(str)** is a PRG which doubles the length of the output by 1 where the key is a binary string.
- **seed()** is a function which actually sets a suitable key as a binary string.
- **prf()** is PRF constructed using OWP. 

In [4]:
def start_session():
    global p
    p = number.getPrime(length)

In [5]:
def sieve(n):
    k = 2
    while(k*k <= n):
        if lst[k] == True : 
            x = k*k
            while x<=n :
                lst[x] = False
                x+=k
        k+=1

n = 10000000
lst = []
for i in range(0,n+1) :
    lst.append( True)
lst2 = []
sieve(n)
for i in range(2,n+1):
    if lst[i] == True:
        lst2.append(i)

In [6]:
def generator():
    global p
    y = p - 1
    lstfact = []
    for i in range(0,len(lst2)):
        if (p - 1) < lst2[i]:
            break
        if (p - 1)%lst2[i] == 0:
            lstfact.append(lst2[i])
    generator = -1
    for j in range(2,p):
        a = False
        for i in lstfact:
            if ((( j**(((p-1)//i))) % p) == 1):
                a = True
                break
        if not a:
            return j

In [7]:
def owp(x):
    global p
    global gp
    return (gp**(int(x,2)))%p
    

In [8]:
def innerproduct(x,r):
    X = x
    R = r 
    result = None
    for bit in range(len(X)):
        if result == None:
            result = int(X[bit])*int(R[bit])
        else :
            result = result ^ (int(X[bit])*int(R[bit]))
    return result

In [9]:
def prg_1(binKey): # str
    # goldreich -levin thm Idea
    # binKey = bin(key)[2:].rjust(length,'0')
    # print(binKey)
    # print(len(binKey))
    assert(len(binKey)%2 == 0)
    l = int(len(binKey)/2)
    x = binKey[:l]
    r = binKey[l:]
    prstring = bin(owp(x))[2:].rjust(int(length),'0') + r + str(innerproduct(x,r))
    # print(prstring)
    # print(len(prstring))
    assert(len(prstring) == len(binKey) + 1)
    return prstring


In [10]:
def prg(key):
    # length doubling prg
    prstring = ''
    lastRStr = key # type = str
    for num in range(len(key)):
        binLast = prg_1(lastRStr) # type = str
        prstring = prstring + binLast[0]
        lastRStr = binLast[1:]
    prstring += lastRStr
    # print(len(key))
    # print(len(prstring))
    assert(len(prstring) == 2 * len(key))
    
    return prstring

In [16]:
def seed():
    global Key
    Key = bin(random.randint(1,p-1))[2:].rjust(length,'0') + bin(random.randint(1,p-1))[2:].rjust(length,'0')

In [17]:
def prf(x):
    global Key
    bin_x = bin(x)[2:].rjust(length,'0')
    CurrKey = Key
    for i in range(len(bin_x)):
        binrandnum = prg(CurrKey)
        if (bin_x[i] == '1'):
            CurrKey = binrandnum[int(len(binrandnum)/2):]
        else:
            CurrKey = binrandnum[:int(len(binrandnum)/2)]
    return int(CurrKey,2)
    

Session starting Code: run it once to start the session. 

In [18]:
start_session()
seed()
gp = generator()

In [20]:
prf(260)

10326